In [1]:
import os
import json
from pycocotools.coco import COCO

data_dir = '/opt/ml/input/data'

In [2]:
###### 두 json 하나의 json으로 combine #####
# 실행시간: 10초
# 사용법: 2가지만 지정해주면됨

# 변경할 json 경로 지정
file_name = 'pseudo.json' #########지정해줄꺼 1: 합칠 json 1##########
target_dir = data_dir + '/' + file_name

sub_file_name = 'train_all_sorted.json' #########지정해줄꺼 2: 합칠 json 1##########
sub_dir = data_dir + '/' + sub_file_name

remove_thr = 0

# 제거할 이미지 파일명 추가: #########(옵션)지정해줄꺼 3##########
exclude = [
    
]
exclude = set(exclude)


target_json = json.load(open(target_dir))
target_coco = COCO(target_dir)
sub_json = json.load(open(sub_dir))
sub_coco = COCO(sub_dir)
print(f'{file_name} 기존 이미지 개수 = {len(target_json["images"])}')
print(f'{file_name} 기존 annotation 개수 = {len(target_json["annotations"])}')

print(f'{sub_file_name} 기존 이미지 개수 = {len(sub_json["images"])}')
print(f'{sub_file_name} 기존 annotation 개수 = {len(sub_json["annotations"])}')

# 새 json 객체 생성
new_json = {
    'info': target_json['info'],
    'licenses': target_json['licenses'],
    'images': [],
    'categories': target_json['categories'],
    'annotations': [],
}
##################################################
img_idx = 0 # img 인덱스 -> 순서대로 0부터 1씩 증가해야함
anno_idx = 0 # ann 인덱스 -> 이미지 상관없이 순서대로 0부터 1씩 증가해야함
print(f'시작 img 인덱스 = {img_idx}')
print(f'시작 anno 인덱스 = {anno_idx}')
removed_annos_cnt = 0
small_removed_annos_cnt = 0
for i in range(len(target_json['images'][:])):
    
    img_id = target_coco.getImgIds(imgIds=i)
    img_info = target_coco.loadImgs(img_id)[0]
    img_name = img_info['file_name']

    ann_ids = target_coco.getAnnIds(imgIds=img_info['id'])
    anns = target_coco.loadAnns(ann_ids)
    anns.sort(key=lambda ann:ann['area'], reverse=True) # area 큰 순

    new_json['images'].append(
        {
            'license':0,
            'url': None,
            'file_name': img_name,
            'height': 512,
            'width': 512,
            'date_captured': None,
            'id': img_idx,
        }
    )
    
    for ann in anns:
        ann['id'] = anno_idx
        ann['image_id'] = img_idx
        anno_idx += 1
        new_json['annotations'].append(ann)

    img_idx += 1

##################################################
print(f'시작 img 인덱스 = {img_idx}')
print(f'시작 anno 인덱스 = {anno_idx}')
for i in range(len(sub_json['images'][:])):
    
    img_id = sub_coco.getImgIds(imgIds=i)
    img_info = sub_coco.loadImgs(img_id)[0]
    img_name = img_info['file_name']
    

    ann_ids = sub_coco.getAnnIds(imgIds=img_info['id'])
    anns = sub_coco.loadAnns(ann_ids)
    anns.sort(key=lambda ann:ann['area'], reverse=True) # area 큰 순

    new_json['images'].append(
        {
            'license':0,
            'url': None,
            'file_name': img_name,
            'height': 512,
            'width': 512,
            'date_captured': None,
            'id': img_idx,
        }
    )
    
    for ann in anns:
        ann['id'] = anno_idx
        ann['image_id'] = img_idx

        if ann['category_id']==0:
            removed_annos_cnt += 1
            continue

        anno_idx += 1
        new_json['annotations'].append(ann)

    img_idx += 1

print(f'마지막 img 인덱스 = {img_idx}')
print(f'마지막 anno 인덱스 = {anno_idx}')

print(f'삭제된 이미지 개수 = {len(exclude)}')
print(f'삭제된 annotation 개수 = {removed_annos_cnt}')

print(f'작아서 삭제된 annotation 개수 = {small_removed_annos_cnt}')

print(f'변경 후 이미지 개수 = {len(new_json["images"])}')
print(f'변경 후 annotation 개수 = {len(new_json["annotations"])}')

loading annotations into memory...
Done (t=0.74s)
creating index...
index created!
loading annotations into memory...
Done (t=5.16s)
creating index...
index created!
pseudo.json 기존 이미지 개수 = 819
pseudo.json 기존 annotation 개수 = 3248
train_all_sorted.json 기존 이미지 개수 = 3272
train_all_sorted.json 기존 annotation 개수 = 26240
시작 img 인덱스 = 0
시작 anno 인덱스 = 0
시작 img 인덱스 = 819
시작 anno 인덱스 = 3248
마지막 img 인덱스 = 4091
마지막 anno 인덱스 = 29488
삭제된 이미지 개수 = 0
삭제된 annotation 개수 = 0
작아서 삭제된 annotation 개수 = 0
변경 후 이미지 개수 = 4091
변경 후 annotation 개수 = 29488


In [3]:
# 저장
save_path = os.path.join(data_dir,f'{file_name.split(".")[0]}_{sub_file_name.split(".")[0]}_combined.json')

with open(save_path,'w') as f:
        json.dump(new_json, f, indent=4)

# coco 포멧 맞는지 확인
demo = COCO(save_path)

loading annotations into memory...
Done (t=7.09s)
creating index...
index created!
